In [1]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN
The
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 3)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [5/200], loss:-1.2479
x:  tensor([[  1.4183, -21.1741,   4.7507]], grad_fn=<AddmmBackward>)
y: tensor([[  1.6349, -21.5724,   5.1058]], grad_fn=<AddmmBackward>)
b: tensor([[  5.3063, -21.0057,   2.3876]], grad_fn=<AddmmBackward>)
epoch [6/200], loss:-1.3121
x:  tensor([[ 6.7524, -9.5143, -0.1142]], grad_fn=<AddmmBackward>)
y: tensor([[ 7.5506, -9.3318, -0.3491]], grad_fn=<AddmmBackward>)
b: tensor([[ 10.1334, -10.8108,  -2.5760]], grad_fn=<AddmmBackward>)
epoch [7/200], loss:-1.4732
x:  tensor([[ 11.3407, -12.8113,   4.8092]], grad_fn=<AddmmBackward>)
y: tensor([[ 11.2764, -13.2232,   5.5986]], grad_fn=<AddmmBackward>)
b: tensor([[ 14.6022, -13.5790,   5.1779]], grad_fn=<AddmmBackward>)
epoch [8/200], loss:-1.3117
x:  tensor([[ 19.6211,   6.7927, -16.7445]], grad_fn=<AddmmBackward>)
y: tensor([[ 19.1744,   6.4551, -17.3446]], grad_fn=<AddmmBackward>)
b: tensor([[ 22.4448,   6.5339, -17.7882]], grad_fn=<AddmmBackward>)
epoch [9/200], loss:-1.6667
x:  tensor([[ 47.6490, -36.8067, -

epoch [41/200], loss:-1.4372
x:  tensor([[ 15.8977,  -0.1407, -12.5772]], grad_fn=<AddmmBackward>)
y: tensor([[ 16.8349,  -1.0653, -12.7193]], grad_fn=<AddmmBackward>)
b: tensor([[ 14.0477,  -2.2587, -12.4918]], grad_fn=<AddmmBackward>)
epoch [42/200], loss:-1.3516
x:  tensor([[  8.5777, -10.5288,   0.4568]], grad_fn=<AddmmBackward>)
y: tensor([[  8.1907, -10.6200,  -0.9159]], grad_fn=<AddmmBackward>)
b: tensor([[  6.4986, -11.2687,  -1.0091]], grad_fn=<AddmmBackward>)
epoch [43/200], loss:-1.4249
x:  tensor([[ 1.3684, -0.5625, -7.6583]], grad_fn=<AddmmBackward>)
y: tensor([[ 1.6399,  0.5029, -6.9560]], grad_fn=<AddmmBackward>)
b: tensor([[ 2.0659, -2.0972, -7.7514]], grad_fn=<AddmmBackward>)
epoch [44/200], loss:-1.5337
x:  tensor([[ -7.0951,  15.9843, -10.6708]], grad_fn=<AddmmBackward>)
y: tensor([[ -6.8243,  16.7186, -10.0107]], grad_fn=<AddmmBackward>)
b: tensor([[ -5.8249,  11.6169, -10.1865]], grad_fn=<AddmmBackward>)
epoch [45/200], loss:-1.3372
x:  tensor([[ 10.8912,  -2.0163,

epoch [76/200], loss:-1.3668
x:  tensor([[ 11.5956, -28.4126, -54.4545]], grad_fn=<AddmmBackward>)
y: tensor([[ 11.4275, -27.4970, -54.9510]], grad_fn=<AddmmBackward>)
b: tensor([[  8.7796, -22.8760, -41.9084]], grad_fn=<AddmmBackward>)
epoch [77/200], loss:-1.4925
x:  tensor([[ -0.7964,  -5.5228, -39.6169]], grad_fn=<AddmmBackward>)
y: tensor([[ -1.4430,  -6.4815, -38.8313]], grad_fn=<AddmmBackward>)
b: tensor([[ -1.4712,  -5.9933, -29.4333]], grad_fn=<AddmmBackward>)
epoch [78/200], loss:-1.4287
x:  tensor([[ -1.6291,  -6.8620, -22.0423]], grad_fn=<AddmmBackward>)
y: tensor([[ -1.3359,  -5.9762, -21.1154]], grad_fn=<AddmmBackward>)
b: tensor([[ -1.0921,  -7.0541, -16.8179]], grad_fn=<AddmmBackward>)
epoch [79/200], loss:-1.5408
x:  tensor([[-12.4235,  -8.7304,  -9.0286]], grad_fn=<AddmmBackward>)
y: tensor([[-12.6642,  -7.9397,  -9.3725]], grad_fn=<AddmmBackward>)
b: tensor([[-9.5463, -7.4639, -7.6248]], grad_fn=<AddmmBackward>)
epoch [80/200], loss:-1.2802
x:  tensor([[ -4.1911,   4

epoch [112/200], loss:-1.3413
x:  tensor([[ 16.1755, -10.8698,   0.7730]], grad_fn=<AddmmBackward>)
y: tensor([[ 15.7295, -10.9831,   2.0454]], grad_fn=<AddmmBackward>)
b: tensor([[10.9050, -8.3142,  0.9165]], grad_fn=<AddmmBackward>)
epoch [113/200], loss:-1.4874
x:  tensor([[14.2713, -0.4382, -7.4271]], grad_fn=<AddmmBackward>)
y: tensor([[13.7037, -0.6168, -6.0678]], grad_fn=<AddmmBackward>)
b: tensor([[ 9.5622, -0.7439, -4.2551]], grad_fn=<AddmmBackward>)
epoch [114/200], loss:-1.3499
x:  tensor([[13.7497,  5.4889,  0.1233]], grad_fn=<AddmmBackward>)
y: tensor([[1.4465e+01, 5.0053e+00, 4.4246e-03]], grad_fn=<AddmmBackward>)
b: tensor([[ 9.5069,  2.0976, -0.2682]], grad_fn=<AddmmBackward>)
epoch [115/200], loss:-1.4891
x:  tensor([[17.5229, 14.9175, -1.7596]], grad_fn=<AddmmBackward>)
y: tensor([[16.9317, 14.7192, -0.3546]], grad_fn=<AddmmBackward>)
b: tensor([[11.3762, 10.6618, -0.7070]], grad_fn=<AddmmBackward>)
epoch [116/200], loss:-1.4846
x:  tensor([[24.3239, 11.3514,  2.6508]

epoch [148/200], loss:-1.4119
x:  tensor([[-0.4092, -1.6738, -4.4984]], grad_fn=<AddmmBackward>)
y: tensor([[-0.5621, -1.0943, -4.5441]], grad_fn=<AddmmBackward>)
b: tensor([[-1.2328, -0.4295, -2.4723]], grad_fn=<AddmmBackward>)
epoch [149/200], loss:-1.3860
x:  tensor([[ 5.1508, 20.2313, -5.7919]], grad_fn=<AddmmBackward>)
y: tensor([[ 4.9914, 21.1324, -6.1029]], grad_fn=<AddmmBackward>)
b: tensor([[ 1.2308, 12.8327, -3.5490]], grad_fn=<AddmmBackward>)
epoch [150/200], loss:-1.4629
x:  tensor([[ 4.1185,  1.8081, -0.9799]], grad_fn=<AddmmBackward>)
y: tensor([[ 3.9387,  2.7945, -1.3056]], grad_fn=<AddmmBackward>)
b: tensor([[ 2.1164,  1.8716, -0.9810]], grad_fn=<AddmmBackward>)
epoch [151/200], loss:-1.5663
x:  tensor([[-5.5155, -0.9850, -8.5696]], grad_fn=<AddmmBackward>)
y: tensor([[-5.2391, -0.0800, -7.8433]], grad_fn=<AddmmBackward>)
b: tensor([[-3.7980, -0.8553, -5.3193]], grad_fn=<AddmmBackward>)
epoch [152/200], loss:-1.2629
x:  tensor([[-10.6960,   2.0760,  -8.7768]], grad_fn=<

epoch [184/200], loss:-1.3479
x:  tensor([[ 8.2516, 24.3927, -2.0300]], grad_fn=<AddmmBackward>)
y: tensor([[ 8.9348, 23.9761, -2.1514]], grad_fn=<AddmmBackward>)
b: tensor([[ 4.9086, 15.1899, -1.3179]], grad_fn=<AddmmBackward>)
epoch [185/200], loss:-1.5129
x:  tensor([[ 8.6718, -3.2053,  9.9873]], grad_fn=<AddmmBackward>)
y: tensor([[ 8.1031, -3.9296,  9.5430]], grad_fn=<AddmmBackward>)
b: tensor([[ 4.2224, -1.9000,  7.6499]], grad_fn=<AddmmBackward>)
epoch [186/200], loss:-1.3144
x:  tensor([[-11.9809,  -5.9194,   1.0790]], grad_fn=<AddmmBackward>)
y: tensor([[-12.4890,  -5.9810,  -0.3418]], grad_fn=<AddmmBackward>)
b: tensor([[-9.4494, -4.6374,  1.1123]], grad_fn=<AddmmBackward>)
epoch [187/200], loss:-1.3759
x:  tensor([[-6.1572, -9.5806,  0.7354]], grad_fn=<AddmmBackward>)
y: tensor([[ -6.6711, -10.3722,   0.3718]], grad_fn=<AddmmBackward>)
b: tensor([[-5.8664, -7.2213,  1.5032]], grad_fn=<AddmmBackward>)
epoch [188/200], loss:-1.3117
x:  tensor([[12.0992,  0.3496,  7.7432]], gra

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [8]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

0
50
1
100
2
150
3
200
4
250
5
300
6
350
7
400
8
450
9
500
10
550
11
600
length  600


In [7]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

675


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time


#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

for i in range(12):
    z = uframes[i]
    z  = cv2.resize(z,(100,100))
    img = uframes[i]
    #print(np.shape(img))
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    print("X: ", temp[0])
    z = clf.predict(temp)
    print('State: ', names[z[0]])
    


X:  [ -4.6092443  -21.59348     -0.71470845]
State:  Just Background
X:  [ -3.9180393 -22.07875    -0.8780521]
State:  Only Tank moving
X:  [ -3.0483484 -22.989056   -0.896033 ]
State:  Only Drone Moving
X:  [ -2.3571453 -23.47433    -1.0593772]
State:  Drone stationary, Tank Stationary
X:  [ -2.4323351 -22.835854   -1.0742056]
State:  Drone Stationary, Tank Reloading
X:  [ -2.6588042 -21.996367   -1.368041 ]
State:  "Drone Stationary, Tank Firing"
X:  [ -3.12954   -22.096378   -2.8093562]
State:  Drone reloading, Tank Stationary
X:  [ -3.642076  -22.83243    -3.2878675]
State:  Drone reloading, Tank Reloading
X:  [ -4.2487235 -23.857761   -2.4440484]
State:  "Drone Reloading, Tank Firing"
X:  [ -4.703061  -23.976633   -1.1704459]
State:  "Drone Firing, Tank Stationary"
X:  [ -4.778252  -23.338156   -1.1852738]
State:  "Drone Firing, Tank Reloading"
X:  [ -5.0047207 -22.498669   -1.4791098]
State:  "Drone Firing, Tank Firing"


In [15]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]
[[ -3.5952842 -15.036567   -1.0762653]]
[0]


In [14]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4